In [1]:
%pip install minio

Note: you may need to restart the kernel to use updated packages.


In [2]:
from minio import Minio

# MinIO connection
client = Minio(
    "minio:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False,
)

bucket_name = "raw"
prefix = "heartbeat/"  # set to "heartbeat/" if you use a subfolder

# List objects
print("Heartbeat files:")
for obj in client.list_objects(bucket_name, prefix=prefix, recursive=True):
    print(obj.object_name)


Heartbeat files:
heartbeat/airflow_time_20251216_205900.txt
heartbeat/airflow_time_20251216_205935.txt
heartbeat/airflow_time_20251216_210110.txt
heartbeat/airflow_time_20251216_210154.txt
heartbeat/airflow_time_20251216_210215.txt


In [3]:
from minio import Minio
from datetime import timezone

client = Minio(
    "minio:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False,
)

bucket_name = "raw"
prefix = ""  # set to "heartbeat/" if applicable

# Collect objects
objects = list(client.list_objects(bucket_name, prefix=prefix, recursive=True))

if not objects:
    raise RuntimeError("No heartbeat files found in raw bucket")

# Select latest by last_modified timestamp
latest = max(objects, key=lambda o: o.last_modified)

print(f"Latest file: {latest.object_name}")
print(f"Last modified: {latest.last_modified.astimezone(timezone.utc)}\n")

# Fetch and print contents
response = client.get_object(bucket_name, latest.object_name)
try:
    content = response.read().decode("utf-8")
    print("File contents:\n")
    print(content)
finally:
    response.close()
    response.release_conn()


Latest file: heartbeat/airflow_time_20251216_210215.txt
Last modified: 2025-12-16 10:02:15.347000+00:00

File contents:

2025-12-16 21:02:15 AEDT

